In [1]:
# -*- coding: utf-8 -*-
# 📦 Импорт необходимых библиотек
import pandas as pd
import numpy as np
from google.colab import drive

# 🔗 Монтируем Google Drive для доступа к файлам
drive.mount('/content/drive')

# 📁 Указываем путь к папке с данными на Google Диске
base_path = '/content/drive/MyDrive/Colab Notebooks/DataSet/'

# 📄 Загружаем исходные CSV-файлы
diaries_df    = pd.read_csv(base_path + 'Self-control diary.csv')  # Дневник самоконтроля
rpe_field_df  = pd.read_csv(base_path + 'RPE FIELD.csv')           # RPE после поля
rpe_gym_df    = pd.read_csv(base_path + 'RPE GYM.csv')             # RPE после зала
gps_2025_df   = pd.read_csv(base_path + 'GPS2025.csv')             # GPS данные за 2025
gps_2024_df   = pd.read_csv(base_path + 'GPS2024.csv')             # GPS данные за 2024

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 🧮 Преобразуем значения в столбце "TOTAL distance (km)" в числовой формат
for gps_df in [gps_2024_df, gps_2025_df]:
    gps_df['TOTAL distance (km)'] = (
        gps_df['TOTAL distance (km)']
        .astype(str)                          # 🔤 Преобразуем в строку (если тип смешанный)
        .str.replace(',', '.', regex=False)   # 🔁 Меняем запятую на точку (европейский формат)
        .astype(float)                         # ➡️ Преобразуем в float
    )

In [3]:
# 📆 Универсальная функция для очистки и стандартизации столбца с датой
def clean_date_column(df, col='Дата'):
    # 🔤 Преобразуем значения в строку и убираем лишние пробелы
    df[col] = df[col].astype(str).str.strip()

    # 🔁 Заменяем точки на слэши (если формат dd.mm.yyyy)
    df[col] = df[col].str.replace('.', '/', regex=False)

    # ✂️ Убираем время (если дата содержит часы/минуты)
    df[col] = df[col].str.split().str[0]

    # 📅 Преобразуем в формат datetime (сначала день, потом месяц)
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')

    return df

In [4]:
# 🚿 Применяем функцию очистки даты ко всем датафреймам
for df in [diaries_df, rpe_gym_df, rpe_field_df, gps_2024_df, gps_2025_df]:
    clean_date_column(df)

In [5]:
# 🔎 Проверяем, сколько строк осталось без корректной даты после преобразования
print("\n📊 Пропущенные даты после преобразования:")

for name, df in zip(
    ['diaries_df', 'rpe_gym_df', 'rpe_field_df', 'gps_2024_df', 'gps_2025_df'],
    [diaries_df, rpe_gym_df, rpe_field_df, gps_2024_df, gps_2025_df]
):
    missing_count = df['Дата'].isna().sum()
    print(f"• {name}: {missing_count} строк без корректной даты")


📊 Пропущенные даты после преобразования:
• diaries_df: 0 строк без корректной даты
• rpe_gym_df: 8 строк без корректной даты
• rpe_field_df: 2 строк без корректной даты
• gps_2024_df: 103 строк без корректной даты
• gps_2025_df: 183 строк без корректной даты


In [6]:
# 🧹 Удаляем строки, где не удалось преобразовать дату (т.е. Дата = NaT)
for df in [diaries_df, rpe_gym_df, rpe_field_df, gps_2024_df, gps_2025_df]:
    df.dropna(subset=['Дата'], inplace=True)

In [7]:
# 🔗 Объединяем GPS-данные за 2024 и 2025 годы в один датафрейм
gps_all = pd.concat([gps_2024_df, gps_2025_df], ignore_index=True)

# 🏷️ Переименовываем колонки с оценкой нагрузки (RPE), чтобы различать зал и поле
rpe_gym_df   = rpe_gym_df.rename(columns={'RPE': 'RPE_GYM'})
rpe_field_df = rpe_field_df.rename(columns={'RPE': 'RPE_FIELD'})

# 🔄 Объединяем все таблицы по ключам "Дата" и "Имя"
merged = diaries_df \
    .merge(rpe_gym_df,   on=['Дата', 'Имя'], how='left') \
    .merge(rpe_field_df, on=['Дата', 'Имя'], how='left') \
    .merge(gps_all,      on=['Дата', 'Имя'], how='left')

# 💾 Сохраняем финальный объединённый датафрейм в Google Drive
merged.to_csv(base_path + 'merged_dataset.csv', index=False, encoding='utf-8-sig')
print("✅ Готово! Файл сохранён как 'merged_dataset.csv'")

# 👀 Просматриваем первые строки и размер таблицы
print(f"📐 Размер итогового датафрейма: {merged.shape}")
display(merged.head())

✅ Готово! Файл сохранён как 'merged_dataset.csv'
📐 Размер итогового датафрейма: (10210, 17)


,Дата,Имя,Часов сна,Качество сна,Стресс,Восстановление,Симптомы заболевания,Мышечные боли,Мотивация,РПЕ_x,РПЕ_y,№ Тренировки,TOTAL distance (km),Sprint Distance (m),PlayerLoad,Unnamed: 6,Unnamed: 7
0,2024-01-11,Aksenov Vladimir,9,3.0,1.0,5.0,Good / Хорошее самочувствие,Nothing / Ничего,5.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-11,Sbotov Ivan,7,3.0,2.0,5.0,Good / Хорошее самочувствие,Nothing / Ничего,5.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-11,Gilev Nikolay,8,3.0,2.0,4.0,Good / Хорошее самочувствие,Nothing / Ничего,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-11,Frolov Ilya,7,3.0,1.0,5.0,Good / Хорошее самочувствие,Nothing / Ничего,5.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-11,Starichenko Vlad,9,3.0,1.0,5.0,NaN,Back (lower back) / Спина (поясница),5.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
